In [1]:
import pandas as pd

from citations_fun.merge_citations import merge_citation_dfs
from citations_fun.get_citation_string import get_citation_str

# load results - datacite, scholex and overton

# nerc_citations_2025/results/latest_results_dataCite.csv
# /home/matnic/Projects/citationNotebook/nerc_citations_2025/results/latest_results_dataCite.csv
# dataCite_df = pd.read_csv("results/latest_results_dataCite.csv")
dataCite_df = pd.read_csv("results/latest_results_dataCite_publicationInfo.csv") # change datacite to write to just latest_results file
scholex_df = pd.read_csv("results/latest_results_scholex.csv")
overton_df = pd.read_csv("results/latest_results_overton.csv")
# merge in a general way

In [2]:
df_list = [dataCite_df, scholex_df, overton_df]

nerc_citations_df = merge_citation_dfs(df_list)

In [ ]:
# get citation string - - takes about 2 hours

nerc_citations_df = get_citation_str(nerc_citations_df)

In [ ]:

# add url to doi in extra columns
nerc_citations_df['data_doi_url'] = 'doi.org/' + nerc_citations_df['data_doi']
# need extra logic here as overton pub_doi column usually just a normal url
nerc_citations_df['publication_doi_url'] = nerc_citations_df['pub_doi'].apply(
    lambda x: f"doi.org/{x}" if x.startswith("10.") else x
)


In [ ]:
# create publicatiYear column from pub_date
def pub_year_splitter(date):
    if date == "Info not given":
        return None
    else:
        try:
            return date.split('/')[2]
        except:
            try:
                return date.split('-')[0]
            except:
                print(date)
                return None

nerc_citations_df['publicationYear'] = nerc_citations_df['pub_date'].apply(pub_year_splitter)

nan
nan


In [ ]:
# map to excected column names for API
# data_Publisher	data_doi	data_Title	data_Authors	relation_type_id	
# publication_doi	publication_type	publication_title	publication_authors	publication_date	citation_event_source	PubCitationStr	
# data_doi_url	publication_doi_url	publicationYear
# nerc_citations_df = pd.read_csv("results/latest_results_nerc_citations.csv")
# old_results = pd.read_csv("/home/matnic/Projects/citationNotebook/Results/v2/latest_results.csv")

# print(old_results.columns)
# print(len(old_results.columns))
# print(nerc_citations_df.columns)
# print(len(nerc_citations_df.columns))

cols = {
    # 'old':'new',
    'relation_type':'relation_type_id', 
    'pub_doi':'publication_doi', 'pub_title':'publication_title', 'pub_date':'publication_date',
    'pub_authors':'publication_authors', 'source_id':'citation_event_source', 'pub_type':'publication_type',
    'pub_citation_str':'PubCitationStr'
}

nerc_citations_df_renamed = nerc_citations_df.rename(columns=cols)



Index(['data_Publisher', 'data_doi', 'data_Title', 'data_Authors',
       'relation_type_id', 'publication_doi', 'publication_type',
       'publication_title', 'publication_authors', 'publication_date',
       'citation_event_source', 'PubCitationStr', 'data_doi_url',
       'publication_doi_url', 'publicationYear'],
      dtype='object')
15
Index(['data_doi', 'data_publisher', 'data_title', 'data_publication_year',
       'data_authors', 'relation_type', 'pub_doi', 'pub_title', 'pub_date',
       'pub_authors', 'source_id', 'pub_publisher', 'pub_type',
       'pub_citation_str', 'pub_doi_stripped'],
      dtype='object')
15


In [ ]:
# write to csv and json file
results_folder_path = "Results/v3/"

latest_file_name_csv = results_folder_path + 'latest_results.csv'
nerc_citations_df.to_csv(latest_file_name_csv, index= False)

# write data to 'latest_results' json file with data publisher as top level key
latest_file_name_json = results_folder_path + 'latest_results.json'

# Group by 'data_Publisher' and convert the DataFrame to a nested dictionary
nested_dict = nerc_citations_df.groupby('data_pub').apply(
    lambda x: x.drop('data_pub', axis=1).to_dict(orient='records')
).to_dict()

# Convert the nested dictionary to a JSON object
import json
json_object = json.dumps(nested_dict)

# Save the JSON object to a file
with open(latest_file_name_json, 'w') as f:
    f.write(json_object)
    

Development stuff below here:

In [ ]:
# code to filter out things like if pub_doi contians "egusphere" etc? - always a conference abstract
count = 0
for pub_doi in nerc_citations_df['pub_doi']:
    if "egusphere" in pub_doi:
        print(pub_doi)
        count =count + 1
print(count)

In [31]:
nerc_citations_df['pub_citation_str'].value_counts()

pub_citation_str
not a doi                                                                                                                                                                                                                                                                                                                                                                                    287
DOI exists but metadata could not be retrieved - contact info@doi.org for help with this                                                                                                                                                                                                                                                                                                     137
DOI not found                                                                                                                                                                                                        

In [ ]:
nerc_citations_df["pub_type"].value_counts()


pub_type
dataset                8729
literature             2281
journal-article        1436
Info not given          906
not a doi               113
Publication             112
posted-content           59
book-chapter              8
Working paper             6
proceedings-article       1
report                    1
other                     1
API request failed        1
Name: count, dtype: int64

0          https://doi.org/10.1016/j.scitotenv.2012.05.023
2                     https://doi.org/10.1002/2015gl065750
4                       https://doi.org/10.17863/cam.20713
6                  https://doi.org/10.1111/1365-2656.12728
8                       https://doi.org/10.5194/cp-2017-18
                               ...                        
24915    https://www.mext.go.jp/content/20230531-mxt_ka...
24916    https://oaarchive.arctic-council.org/handle/11...
24917    https://www.miljodirektoratet.no/publikasjoner...
24918    https://nilu.brage.unit.no/nilu-xmlui/handle/1...
24919                  https://idrc-crdi.ca/en/node/123325
Name: pub_doi, Length: 13654, dtype: object

In [ ]:
# # take data_doi and pub_doi columns from each df, concatenate vertically
# doi_df = pd.concat([scholex_df[['data_doi', 'pub_doi']], dataCite_df[['data_doi', 'pub_doi']], overton_df[['data_doi', 'pub_doi']]], ignore_index=True)
# doi_df_unique = doi_df.drop_duplicates(subset=['data_doi', 'pub_doi'])
# print(len(doi_df))
# print(len(doi_df_unique))